In [1]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('train_set.csv')
df

,id_img,path,label
0,0,../data/images/train/angry\0.jpg,angry
1,1,../data/images/train/angry\1.jpg,angry
2,10,../data/images/train/angry\10.jpg,angry
3,10002,../data/images/train/angry\10002.jpg,angry
4,10016,../data/images/train/angry\10016.jpg,angry
...,...,...,...
28816,9969,../data/images/train/surprise\9969.jpg,surprise
28817,9985,../data/images/train/surprise\9985.jpg,surprise
28818,9990,../data/images/train/surprise\9990.jpg,surprise
28819,9992,../data/images/train/surprise\9992.jpg,surprise


In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['target'] = label_encoder.fit_transform(df['label'])

In [4]:
df['path'] = df['path'].str.replace('\\', '/')
df['path'] = df['path'].str.replace('../data/images/train/', './data/images/train/')

def cargar_y_convertir_imagen(ruta_imagen):
    image = cv2.imread(ruta_imagen, 0)
    image_array = np.array(image)
    return image_array

df['imagenes'] =df['path'].apply(cargar_y_convertir_imagen)

In [5]:
df['imagenes'] = df['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 1))

# Separar características (X) y etiquetas (y)
X = np.stack(df['imagenes'].to_numpy())
y = df['target']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [10]:
X = X / 255.0

# Construir el modelo de red neuronal
modelo_nn = Sequential()
modelo_nn.add(Flatten(input_shape=(48, 48, 1)))  # Capa de aplanamiento para convertir la imagen a un vector
modelo_nn.add(Dense(128, activation='relu'))
modelo_nn.add(Dense(7, activation='softmax'))

# Compilar el modelo
modelo_nn.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Imprimir el resumen del modelo
modelo_nn.summary()

# Definir EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

# Entrenar el modelo con EarlyStopping
historia = modelo_nn.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 2304)              0         
                                                                 
 dense_4 (Dense)             (None, 128)               295040    
                                                                 
 dense_5 (Dense)             (None, 7)                 903       
                                                                 
Total params: 295943 (1.13 MB)
Trainable params: 295943 (1.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
721/721 [==============================] - 2s 3ms/step - loss: 1.6098 - accuracy: 0.3289 - val_loss: 7.4391 - val_accuracy: 1.7346e-04
Epoch 2/100
721/721 [==============================] - 2s 2ms/step - loss: 1.5413 - accuracy: 0.3705 - val_loss: 7.0816 - val_accuracy:

In [15]:
from tensorflow.keras.models import save_model, load_model
modelo_nn.save('modelo_nn.h5')

c:\Users\migue\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
test = pd.read_csv('test_set.csv')
test

,id_img
0,10052
1,10065
2,10079
3,10095
4,10121
...,...
7061,9806
7062,9830
7063,9853
7064,9878


In [12]:
test['path'] = test['id_img'].apply(lambda x: f'./data/images/test/{x}.jpg')

In [13]:
test['imagenes'] = test['path'].apply(cargar_y_convertir_imagen)

In [14]:
test.head()

,id_img,path,imagenes
0,10052,./data/images/test/10052.jpg,"[[58, 66, 70, 77, 117, 154, 137, 108, 76, 70, ..."
1,10065,./data/images/test/10065.jpg,"[[23, 26, 21, 9, 6, 19, 33, 11, 3, 63, 89, 73,..."
2,10079,./data/images/test/10079.jpg,"[[201, 182, 182, 184, 205, 204, 203, 220, 223,..."
3,10095,./data/images/test/10095.jpg,"[[93, 86, 78, 78, 80, 92, 109, 99, 104, 107, 1..."
4,10121,./data/images/test/10121.jpg,"[[11, 6, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 2, 3..."


In [16]:
final_model = load_model('modelo_nn.h5')

In [17]:
X_test = np.stack(test['imagenes'].apply(lambda x: np.array(x).reshape(48, 48, 1)) / 255.0)

# Realizar predicciones en el conjunto de prueba utilizando el modelo cargado
y_pred = final_model.predict(X_test)

# Convertir las predicciones a etiquetas (usando argmax para obtener la clase con mayor probabilidad)
y_pred_labels = np.argmax(y_pred, axis=1)

# Puedes agregar las predicciones de vuelta al DataFrame 'test' si es necesario
test['predicted_label'] = y_pred_labels

221/221 [==============================] - 0s 859us/step


In [19]:
test['label'] = label_encoder.inverse_transform(y_pred_labels)

In [20]:
test

,id_img,path,imagenes,predicted_label,label
0,10052,./data/images/test/10052.jpg,"[[58, 66, 70, 77, 117, 154, 137, 108, 76, 70, ...",0,angry
1,10065,./data/images/test/10065.jpg,"[[23, 26, 21, 9, 6, 19, 33, 11, 3, 63, 89, 73,...",2,fear
2,10079,./data/images/test/10079.jpg,"[[201, 182, 182, 184, 205, 204, 203, 220, 223,...",3,happy
3,10095,./data/images/test/10095.jpg,"[[93, 86, 78, 78, 80, 92, 109, 99, 104, 107, 1...",0,angry
4,10121,./data/images/test/10121.jpg,"[[11, 6, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 2, 3...",2,fear
...,...,...,...,...,...
7061,9806,./data/images/test/9806.jpg,"[[255, 253, 255, 255, 253, 255, 255, 250, 254,...",2,fear
7062,9830,./data/images/test/9830.jpg,"[[84, 71, 70, 68, 60, 47, 53, 45, 51, 58, 56, ...",0,angry
7063,9853,./data/images/test/9853.jpg,"[[250, 253, 253, 252, 252, 252, 253, 251, 234,...",0,angry
7064,9878,./data/images/test/9878.jpg,"[[228, 224, 227, 202, 45, 8, 6, 8, 15, 21, 19,...",4,neutral


In [21]:
submission = pd.DataFrame({'id_img': test['id_img'], 'label': test['label']})

submission.to_csv('submission.csv', index=False)

In [22]:
submission

,id_img,label
0,10052,angry
1,10065,fear
2,10079,happy
3,10095,angry
4,10121,fear
...,...,...
7061,9806,fear
7062,9830,angry
7063,9853,angry
7064,9878,neutral
